In [5]:
import networkx as nx
import pandas as pd
from node2vec import Node2Vec
import node2vec

In [21]:
from create_graph import graph
from node2vec_embed import node_2vec



In [4]:
himgraph = graph()

In [7]:
df_features = pd.read_csv('/home/lxu/Downloads/transformed-features.tsv',sep ='\t',low_memory=False)
df_features.head()

,compound_id,compound_name,disease_id,disease_name,category,status,prior_prob,prior_logit,degree_CbG,degree_CcSE,...,dwpc_CuGaDuGaD,dwpc_CuGcG<rGaD,dwpc_CuGcGiGaD,dwpc_CuGdDpSpD,dwpc_CuGr>GbCtD,dwpc_CuGuCrCtD,dwpc_CuGuCtD,dwpc_CuGuCtDrD,dwpc_CuGuCuGaD,dwpc_CuGuDuGaD
0,DB01048,Abacavir,DOID:10652,Alzheimer's disease,NaN,0,0.004753,-5.344,-0.2955,0.8146,...,-0.5028,-0.4211,-0.45810,-0.53990,-0.357,-0.3849,-0.375,-0.5642,-0.5555,-0.4952
1,DB05812,Abiraterone,DOID:10652,Alzheimer's disease,NaN,0,0.004753,-5.344,0.5495,0.5554,...,-0.5028,-0.4211,-0.45810,-0.53990,-0.357,-0.3849,-0.375,-0.5642,-0.5555,-0.4952
2,DB00659,Acamprosate,DOID:10652,Alzheimer's disease,NaN,0,0.004753,-5.344,0.9053,1.0220,...,-0.5028,-0.4211,-0.45810,-0.53990,-0.357,-0.3849,-0.375,-0.5642,-0.5555,-0.4952
3,DB00284,Acarbose,DOID:10652,Alzheimer's disease,NaN,0,0.004753,-5.344,0.4328,0.3460,...,0.4304,0.6755,-0.07508,-0.08597,-0.357,-0.3849,-0.375,-0.5642,0.8816,0.6339
4,DB01193,Acebutolol,DOID:10652,Alzheimer's disease,NaN,0,0.004753,-5.344,0.1403,0.7966,...,-0.5028,-0.4211,-0.45810,-0.53990,-0.357,-0.3849,-0.375,-0.5642,-0.5555,-0.4952


In [8]:
df_positive = df_features.loc[df_features['status']== 1][0:10]
len(df_positive)

10

In [9]:
df_negative = df_features.loc[df_features['status']== 0][0:20]
len(df_negative)

20

In [10]:
subgraph_nodeslist=[]
for index,row in df_positive.iterrows():
    for path in nx.all_simple_paths(himgraph, source='Compound::'+row['compound_id'], target='Disease::'+row['disease_id'],cutoff=3):
        #print(path)
        subgraph_nodeslist=subgraph_nodeslist+path
for i,r in df_negative.iterrows():
     for path in nx.all_simple_paths(himgraph, source='Compound::'+r['compound_id'], target='Disease::'+r['disease_id'],cutoff=3):
        #print(path)
        subgraph_nodeslist=subgraph_nodeslist+path   
subgraph_nodes_list = list(set(subgraph_nodeslist))
subgraph_nodes_list


['Side Effect::C0575081',
 'Compound::DB00448',
 'Gene::5357',
 'Compound::DB00771',
 'Gene::7161',
 'Side Effect::C0034063',
 'Gene::8943',
 'Gene::1937',
 'Symptom::D004415',
 'Gene::2634',
 'Gene::2944',
 'Side Effect::C0041969',
 'Side Effect::C0151888',
 'Compound::DB00787',
 'Compound::DB00459',
 'Gene::22841',
 'Gene::3479',
 'Gene::1573',
 'Gene::23089',
 'Side Effect::C0034155',
 'Side Effect::C0020651',
 'Symptom::D008569',
 'Side Effect::C0151827',
 'Anatomy::UBERON:0002707',
 'Gene::169792',
 'Gene::3593',
 'Gene::7381',
 'Gene::1020',
 'Gene::11255',
 'Gene::3557',
 'Gene::11254',
 'Side Effect::C0242429',
 'Gene::3394',
 'Gene::276',
 'Gene::55244',
 'Compound::DB00507',
 'Gene::8973',
 'Side Effect::C0043037',
 'Gene::965',
 'Gene::858',
 'Gene::1548',
 'Gene::2590',
 'Gene::1006',
 'Gene::150962',
 'Gene::2904',
 'Gene::2879',
 'Side Effect::C0278134',
 'Gene::4170',
 'Gene::8884',
 'Gene::138162',
 'Side Effect::C0022107',
 'Gene::1134',
 'Gene::1808',
 'Side Effect::C

In [14]:
import simplejson
f = open('subgraph_nodes', 'w')
simplejson.dump(subgraph_nodes_list, f)
f.close()

In [15]:
test_graph = himgraph.subgraph(subgraph_nodes_list)
test_graph

In [17]:
positive_list=[]
positve_label=[1,1,1,1,1,1,1,1,1,1]
for _,row in df_positive[['compound_id','disease_id']].iterrows():
    positive_list.append(['Compound::'+row['compound_id'],'Disease::'+row['disease_id']])
negative_list=[]
negative_label=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for _,row in df_negative[['compound_id','disease_id']].iterrows():
    negative_list.append(['Compound::'+row['compound_id'],'Disease::'+row['disease_id']])

In [18]:
ff= open('positive_list', 'w')
simplejson.dump(positive_list, ff)
ff.close()

In [28]:
fff = open('negative_list','w')
simplejson.dump(negative_list, fff)
fff.close()

In [23]:
model= node_2vec(test_graph)


Computing transition probabilities: 100%|██████████| 2450/2450 [03:27<00:00, 11.81it/s]


In [11]:
pos_edges=edge_vec(model,positive_list)
neg_edges = edge_vec(model,negative_list)

NameError: name 'node2vec' is not defined

In [24]:
pair_list=positive_list
edge_vecs = []
edges_embs = node2vec.edges.HadamardEmbedder(keyed_vectors=model.wv)
for i in pair_list:
        node1=i[0]
        node2=i[1]
        vec= edges_embs[(node1,node2)].tolist()
        edge_vecs.append(vec)
print(edge_vecs)

[[0.846593976020813, -13.815777778625488, -0.011118775233626366, 1.07418954372406, -0.09448432922363281, 1.8443067073822021, -2.0249204635620117, 5.752410411834717, -2.1607422828674316, 2.921407699584961, 1.8580763339996338, 0.3552554249763489, 0.6023368239402771, -1.8934861421585083, 17.67643928527832, -14.602627754211426, 2.1517043113708496, -1.2976638078689575, 0.07439342141151428, -2.4483261108398438, 10.270410537719727, -14.895326614379883, -1.9914565086364746, 11.679080963134766, -0.07125586271286011, 4.973766326904297, -20.3543643951416, 0.9963293075561523, -0.12995925545692444, -0.34238168597221375, 14.92921257019043, 0.3077191710472107, -2.1670033931732178, -0.565977156162262, -4.785907745361328, 3.9889063835144043, -9.387585639953613, -90.8578109741211, 3.08223557472229, -0.08144631236791611, 0.44450464844703674, -23.528228759765625, -1.5201694965362549, 2.0799808502197266, -0.8673338294029236, -0.0953206941485405, -4.94611120223999, -6.0795722007751465, -9.005935668945312, -

In [25]:
npair_list=negative_list
nedge_vecs = []
edges_embs = node2vec.edges.HadamardEmbedder(keyed_vectors=model.wv)
for i in npair_list:
        node1=i[0]
        node2=i[1]
        vec= edges_embs[(node1,node2)].tolist()
        nedge_vecs.append(vec)


In [26]:
import numpy as np
train_x= edge_vecs[0:5]+nedge_vecs[0:15]


train_y= positve_label[0:5]+negative_label[0:15]

test_x= edge_vecs[5:]+nedge_vecs[15:]
test_y= positve_label[5:]+negative_label[15:]

In [41]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0,solver='lbfgs',
tol=1e-8).fit(train_x,train_y)

In [42]:
clf.score(test_x,test_y)

0.6

In [43]:
from sklearn.metrics import roc_auc_score
roc=roc_auc_score(test_y, clf.predict_proba(test_x)[:,1])
roc

0.36